# Importaciones necesarias

In [ ]:
!pip install tensorflow pillow
!pip install -q mediapy
!pip install ultralytics

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import mediapy as media
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO
from IPython.display import Image as image_show

# Función para procesar videos

In [ ]:
# Función para procesar un video:
def video_processor(filename_in, filename_out, process_func, max_time=10, **kwargs):
    # Abrir el video de entrada para lectura
    with media.VideoReader(filename_in) as r:
        # Crear un archivo de video de salida
        with media.VideoWriter(filename_out, shape=r.shape, fps=r.fps, bps=r.bps) as w:
            count = 0  # Inicializar contador de fotogramas

            # Iterar sobre cada imagen (fotograma) del video
            for image in r:
                image = media.to_uint8(image)  # Convertir la imagen a formato flotante

                # Procesar la imagen utilizando la función dada
                processed_image = process_func(image, **kwargs)

                if processed_image is not None:
                  # Añadir la imagen procesada al video de salida
                  w.add_image(processed_image)

                # Incrementar el contador de fotogramas
                count += 1

                # Detener el proceso si se alcanza el tiempo máximo
                if count >= max_time * r.fps:
                    break

# EfficientNetB0

In [ ]:
# Cargar el modelo preentrenado EfficientNetB0
model = EfficientNetB0(weights='imagenet')

def classify_image(img):
    # Si se proporciona una ruta de archivo, cargar la imagen
    if isinstance(img, str):
        img = Image.open(img)
    # Asegurarse de que estás trabajando con una copia del array
    img_array = np.array(img.copy())

    # Redimensionar la imagen
    img_array = cv2.resize(img_array, (224, 224))  # Tamaño de entrada para EfficientNetB0

    # Convertir la imagen a un array y procesarla
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Realizar la predicción con el modelo
    predictions = model.predict(img_array)

    # Decodificar y retornar las predicciones
    return decode_predictions(predictions, top=1)[0]

In [ ]:
def predict_object(image_path):

  predictions = classify_image(image_path)

  # Mostrar las predicciones
  for i, (imagenet_id, label, score) in enumerate(predictions):
      print(f"{i + 1}: {label} ({score:.2f})")

In [ ]:
file_in = '/content/drive/MyDrive/computer_vision/Unidad 1/video_bicicleta_recortado.mp4'
file_out = '/content/drive/MyDrive/computer_vision/Unidad 1/video_bicicleta_procesado.mp4'

# Llamar a la función para procesar el video
video_processor(file_in, file_out, predict_object)

# Yolo

In [ ]:
# Load the pre-trained YOLOv8 model
model_YOLO = YOLO('yolov8n')  # This will automatically download the model weights

In [ ]:
# Train the model using the 'coco128.yaml' dataset for 3 epochs
model_YOLO.train(data='coco128.yaml', epochs=3)

In [ ]:
def process_YOLO(image):
    # Realizar inferencia
    results = model(image)

    # Imagen a devolver
    annotated_frame = image

    # Iterar sobre las bounding boxes obtenidas
    for box in results[0].boxes:
        # Extraer los datos de la caja del tensor
        x1, y1, x2, y2, confidence, cls = box.data[0]

        # Obtener el nombre de la clase
        class_name = model.names[int(cls)]

        # Si la clase detectada es 'bicycle', crear bounding box
        if class_name == 'bicycle':
            # Imprimir los resultados en un formato legible
            print(f"Confianza de la clase: {confidence:.2f}")

            # Dibujar bounding box en el marco anotado
            annotated_frame = cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Agregar etiqueta de la clase al bounding box
            cv2.putText(annotated_frame, class_name, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return annotated_frame

In [ ]:
file_in = '/content/drive/MyDrive/computer_vision/Unidad 1/video_bicicleta_recortado.mp4'
file_out = '/content/drive/MyDrive/computer_vision/Unidad 1/video_bicicleta_procesado.mp4'

# Llamar a la función para procesar el video
video_processor(file_in, file_out, process_YOLO)
# Mostrar el video resultante
media.show_video(media.read_video(file_out), fps=30)


0: 384x640 3 persons, 1 bicycle, 7 cars, 1 motorcycle, 1 bench, 40.6ms
Speed: 1.8ms preprocess, 40.6ms inference, 7.4ms postprocess per image at shape (1, 3, 384, 640)
Confianza de la clase: 0.28

0: 384x640 5 persons, 7 cars, 1 motorcycle, 19.5ms
Speed: 2.4ms preprocess, 19.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 6 cars, 1 motorcycle, 19.1ms
Speed: 1.3ms preprocess, 19.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 7 cars, 1 motorcycle, 14.4ms
Speed: 1.3ms preprocess, 14.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8 cars, 1 motorcycle, 20.7ms
Speed: 1.3ms preprocess, 20.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 bicycle, 7 cars, 1 motorcycle, 19.4ms
Speed: 1.3ms preprocess, 19.4ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
Confianza de la clase: 0.32

0: 384x640 5 perso

This browser does not support the video tag.
